# Testing basic tools in documentation


[x] Add    
[x] Get


In [1]:
import molsysmt as msm

In [3]:
item = msm.convert('1L2Y', 'mmcif.PdbxContainers.DataContainer')

In [19]:
item.exists('entry')

True

# entity info

In [37]:
item.getObj('entity')

DataCategory(name:'entity'
attrbuteList: ['id', 'type', 'src_method', 'pdbx_description', 'formula_weight', 'pdbx_number_of_molecules', 'pdbx_ec', 'pdbx_mutation', 'pdbx_fragment', 'details']
Data: [[1, 'polymer', 'syn', 'TC5b', 2171.413, 1, '?', '?', '?', '?']]
)

In [38]:
item.getObj('entity_poly')

DataCategory(name:'entity_poly'
attrbuteList: ['entity_id', 'type', 'nstd_linkage', 'nstd_monomer', 'pdbx_seq_one_letter_code', 'pdbx_seq_one_letter_code_can', 'pdbx_strand_id', 'pdbx_target_identifier']
Data: [[1, 'polypeptide(L)', 'no', 'no', 'NLYIQWLKDGGPSSGRPPPS', 'NLYIQWLKDGGPSSGRPPPS', 'A', '?']]
)

In [39]:
item.getObj('entity_poly_seq')

DataCategory(name:'entity_poly_seq'
attrbuteList: ['entity_id', 'num', 'mon_id', 'hetero']
Data: [[1, 1, 'ASN', 'n'], [1, 2, 'LEU', 'n'], [1, 3, 'TYR', 'n'], [1, 4, 'ILE', 'n'], [1, 5, 'GLN', 'n'], [1, 6, 'TRP', 'n'], [1, 7, 'LEU', 'n'], [1, 8, 'LYS', 'n'], [1, 9, 'ASP', 'n'], [1, 10, 'GLY', 'n'], [1, 11, 'GLY', 'n'], [1, 12, 'PRO', 'n'], [1, 13, 'SER', 'n'], [1, 14, 'SER', 'n'], [1, 15, 'GLY', 'n'], [1, 16, 'ARG', 'n'], [1, 17, 'PRO', 'n'], [1, 18, 'PRO', 'n'], [1, 19, 'PRO', 'n'], [1, 20, 'SER', 'n']]
)

In [41]:
item.getObj('chem_comp')

DataCategory(name:'chem_comp'
attrbuteList: ['id', 'type', 'mon_nstd_flag', 'name', 'pdbx_synonyms', 'formula', 'formula_weight']
Data: [['ARG', 'L-peptide linking', 'y', 'ARGININE', '?', 'C6 H15 N4 O2 1', 175.209], ['ASN', 'L-peptide linking', 'y', 'ASPARAGINE', '?', 'C4 H8 N2 O3', 132.118], ['ASP', 'L-peptide linking', 'y', 'ASPARTIC ACID', '?', 'C4 H7 N O4', 133.103], ['GLN', 'L-peptide linking', 'y', 'GLUTAMINE', '?', 'C5 H10 N2 O3', 146.144], ['GLY', 'peptide linking', 'y', 'GLYCINE', '?', 'C2 H5 N O2', 75.067], ['ILE', 'L-peptide linking', 'y', 'ISOLEUCINE', '?', 'C6 H13 N O2', 131.173], ['LEU', 'L-peptide linking', 'y', 'LEUCINE', '?', 'C6 H13 N O2', 131.173], ['LYS', 'L-peptide linking', 'y', 'LYSINE', '?', 'C6 H15 N2 O2 1', 147.195], ['PRO', 'L-peptide linking', 'y', 'PROLINE', '?', 'C5 H9 N O2', 115.13], ['SER', 'L-peptide linking', 'y', 'SERINE', '?', 'C3 H7 N O3', 105.093], ['TRP', 'L-peptide linking', 'y', 'TRYPTOPHAN', '?', 'C11 H12 N2 O2', 204.225], ['TYR', 'L-peptide li

In [42]:
item.getObj('pdbx_poly_seq_scheme')

DataCategory(name:'pdbx_poly_seq_scheme'
attrbuteList: ['asym_id', 'entity_id', 'seq_id', 'mon_id', 'ndb_seq_num', 'pdb_seq_num', 'auth_seq_num', 'pdb_mon_id', 'auth_mon_id', 'pdb_strand_id', 'pdb_ins_code', 'hetero']
Data: [['A', 1, 1, 'ASN', 1, 1, 1, 'ASN', 'ASN', 'A', '.', 'n'], ['A', 1, 2, 'LEU', 2, 2, 2, 'LEU', 'LEU', 'A', '.', 'n'], ['A', 1, 3, 'TYR', 3, 3, 3, 'TYR', 'TYR', 'A', '.', 'n'], ['A', 1, 4, 'ILE', 4, 4, 4, 'ILE', 'ILE', 'A', '.', 'n'], ['A', 1, 5, 'GLN', 5, 5, 5, 'GLN', 'GLN', 'A', '.', 'n'], ['A', 1, 6, 'TRP', 6, 6, 6, 'TRP', 'TRP', 'A', '.', 'n'], ['A', 1, 7, 'LEU', 7, 7, 7, 'LEU', 'LEU', 'A', '.', 'n'], ['A', 1, 8, 'LYS', 8, 8, 8, 'LYS', 'LYS', 'A', '.', 'n'], ['A', 1, 9, 'ASP', 9, 9, 9, 'ASP', 'ASP', 'A', '.', 'n'], ['A', 1, 10, 'GLY', 10, 10, 10, 'GLY', 'GLY', 'A', '.', 'n'], ['A', 1, 11, 'GLY', 11, 11, 11, 'GLY', 'GLY', 'A', '.', 'n'], ['A', 1, 12, 'PRO', 12, 12, 12, 'PRO', 'PRO', 'A', '.', 'n'], ['A', 1, 13, 'SER', 13, 13, 13, 'SER', 'SER', 'A', '.', 'n'], ['A',

In [45]:
item.getObj('database_PDB_matrix')

DataCategory(name:'database_PDB_matrix'
attrbuteList: ['entry_id', 'origx[1][1]', 'origx[1][2]', 'origx[1][3]', 'origx[2][1]', 'origx[2][2]', 'origx[2][3]', 'origx[3][1]', 'origx[3][2]', 'origx[3][3]', 'origx_vector[1]', 'origx_vector[2]', 'origx_vector[3]']
Data: [['1L2Y', 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]]
)

In [52]:
item.getObj('pdbx_struct_assembly')

DataCategory(name:'pdbx_struct_assembly'
attrbuteList: ['id', 'details', 'method_details', 'oligomeric_details', 'oligomeric_count']
Data: [[1, 'author_defined_assembly', '?', 'monomeric', 1]]
)

In [53]:
item.getObj('pdbx_struct_assembly_gen')

DataCategory(name:'pdbx_struct_assembly_gen'
attrbuteList: ['assembly_id', 'oper_expression', 'asym_id_list']
Data: [[1, 1, 'A']]
)

In [55]:
item.getObj('pdbx_struct_oper_list')

DataCategory(name:'pdbx_struct_oper_list'
attrbuteList: ['id', 'type', 'name', 'symmetry_operation', 'matrix[1][1]', 'matrix[1][2]', 'matrix[1][3]', 'vector[1]', 'matrix[2][1]', 'matrix[2][2]', 'matrix[2][3]', 'vector[2]', 'matrix[3][1]', 'matrix[3][2]', 'matrix[3][3]', 'vector[3]']
Data: [[1, 'identity operation', '1_555', '?', 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]
)

In [56]:
item.getObj('struct_conf')

DataCategory(name:'struct_conf'
attrbuteList: ['conf_type_id', 'id', 'pdbx_PDB_helix_id', 'beg_label_comp_id', 'beg_label_asym_id', 'beg_label_seq_id', 'pdbx_beg_PDB_ins_code', 'end_label_comp_id', 'end_label_asym_id', 'end_label_seq_id', 'pdbx_end_PDB_ins_code', 'beg_auth_comp_id', 'beg_auth_asym_id', 'beg_auth_seq_id', 'end_auth_comp_id', 'end_auth_asym_id', 'end_auth_seq_id', 'pdbx_PDB_helix_class', 'details', 'pdbx_PDB_helix_length']
Data: [['HELX_P', 'HELX_P1', 1, 'ASN', 'A', 1, '?', 'ASP', 'A', 9, '?', 'ASN', 'A', 1, 'ASP', 'A', 9, 1, '?', 9], ['HELX_P', 'HELX_P2', 2, 'GLY', 'A', 10, '?', 'GLY', 'A', 15, '?', 'GLY', 'A', 10, 'GLY', 'A', 15, 5, '?', 6]]
)

In [57]:
item.getObj('struct_conf_type')

DataCategory(name:'struct_conf_type'
attrbuteList: ['id', 'criteria', 'reference']
Data: [['HELX_P', '?', '?']]
)

In [69]:
item.getObj('chem_comp_atom')

DataCategory(name:'chem_comp_atom'
attrbuteList: ['comp_id', 'atom_id', 'type_symbol', 'pdbx_aromatic_flag', 'pdbx_stereo_config', 'pdbx_ordinal']
Data: [['ARG', 'N', 'N', 'N', 'N', 1], ['ARG', 'CA', 'C', 'N', 'S', 2], ['ARG', 'C', 'C', 'N', 'N', 3], ['ARG', 'O', 'O', 'N', 'N', 4], ['ARG', 'CB', 'C', 'N', 'N', 5], ['ARG', 'CG', 'C', 'N', 'N', 6], ['ARG', 'CD', 'C', 'N', 'N', 7], ['ARG', 'NE', 'N', 'N', 'N', 8], ['ARG', 'CZ', 'C', 'N', 'N', 9], ['ARG', 'NH1', 'N', 'N', 'N', 10], ['ARG', 'NH2', 'N', 'N', 'N', 11], ['ARG', 'OXT', 'O', 'N', 'N', 12], ['ARG', 'H', 'H', 'N', 'N', 13], ['ARG', 'H2', 'H', 'N', 'N', 14], ['ARG', 'HA', 'H', 'N', 'N', 15], ['ARG', 'HB2', 'H', 'N', 'N', 16], ['ARG', 'HB3', 'H', 'N', 'N', 17], ['ARG', 'HG2', 'H', 'N', 'N', 18], ['ARG', 'HG3', 'H', 'N', 'N', 19], ['ARG', 'HD2', 'H', 'N', 'N', 20], ['ARG', 'HD3', 'H', 'N', 'N', 21], ['ARG', 'HE', 'H', 'N', 'N', 22], ['ARG', 'HH11', 'H', 'N', 'N', 23], ['ARG', 'HH12', 'H', 'N', 'N', 24], ['ARG', 'HH21', 'H', 'N', 'N',

In [62]:
item.getObj('chem_comp_bond')

DataCategory(name:'chem_comp_bond'
attrbuteList: ['comp_id', 'atom_id_1', 'atom_id_2', 'value_order', 'pdbx_aromatic_flag', 'pdbx_stereo_config', 'pdbx_ordinal']
Data: [['ARG', 'N', 'CA', 'sing', 'N', 'N', 1], ['ARG', 'N', 'H', 'sing', 'N', 'N', 2], ['ARG', 'N', 'H2', 'sing', 'N', 'N', 3], ['ARG', 'CA', 'C', 'sing', 'N', 'N', 4], ['ARG', 'CA', 'CB', 'sing', 'N', 'N', 5], ['ARG', 'CA', 'HA', 'sing', 'N', 'N', 6], ['ARG', 'C', 'O', 'doub', 'N', 'N', 7], ['ARG', 'C', 'OXT', 'sing', 'N', 'N', 8], ['ARG', 'CB', 'CG', 'sing', 'N', 'N', 9], ['ARG', 'CB', 'HB2', 'sing', 'N', 'N', 10], ['ARG', 'CB', 'HB3', 'sing', 'N', 'N', 11], ['ARG', 'CG', 'CD', 'sing', 'N', 'N', 12], ['ARG', 'CG', 'HG2', 'sing', 'N', 'N', 13], ['ARG', 'CG', 'HG3', 'sing', 'N', 'N', 14], ['ARG', 'CD', 'NE', 'sing', 'N', 'N', 15], ['ARG', 'CD', 'HD2', 'sing', 'N', 'N', 16], ['ARG', 'CD', 'HD3', 'sing', 'N', 'N', 17], ['ARG', 'NE', 'CZ', 'sing', 'N', 'N', 18], ['ARG', 'NE', 'HE', 'sing', 'N', 'N', 19], ['ARG', 'CZ', 'NH1', 'si

In [65]:
item.getObj('atom_sites')

DataCategory(name:'atom_sites'
attrbuteList: ['entry_id', 'fract_transf_matrix[1][1]', 'fract_transf_matrix[1][2]', 'fract_transf_matrix[1][3]', 'fract_transf_matrix[2][1]', 'fract_transf_matrix[2][2]', 'fract_transf_matrix[2][3]', 'fract_transf_matrix[3][1]', 'fract_transf_matrix[3][2]', 'fract_transf_matrix[3][3]', 'fract_transf_vector[1]', 'fract_transf_vector[2]', 'fract_transf_vector[3]']
Data: [['1L2Y', 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]]
)

In [66]:
item.getObj('atom_type')

DataCategory(name:'atom_type'
attrbuteList: ['symbol']
Data: [['C'], ['H'], ['N'], ['O']]
)

In [75]:
datacategory_atom_site = item.getObj('atom_site')

In [104]:
aa = { jj:ii for ii,jj in enumerate(datacategory_atom_site.getAttributeList())}

In [105]:
aa

{'group_PDB': 0,
 'id': 1,
 'type_symbol': 2,
 'label_atom_id': 3,
 'label_alt_id': 4,
 'label_comp_id': 5,
 'label_asym_id': 6,
 'label_entity_id': 7,
 'label_seq_id': 8,
 'pdbx_PDB_ins_code': 9,
 'Cartn_x': 10,
 'Cartn_y': 11,
 'Cartn_z': 12,
 'occupancy': 13,
 'B_iso_or_equiv': 14,
 'pdbx_formal_charge': 15,
 'auth_seq_id': 16,
 'auth_comp_id': 17,
 'auth_asym_id': 18,
 'auth_atom_id': 19,
 'pdbx_PDB_model_num': 20}

In [116]:
data = datacategory_atom_site.data

In [119]:
data[1][15]

'?'

In [121]:
for ii in item.getObj('chem_comp_atom'):
    print(ii)

['ARG', 'N', 'N', 'N', 'N', 1]
['ARG', 'CA', 'C', 'N', 'S', 2]
['ARG', 'C', 'C', 'N', 'N', 3]
['ARG', 'O', 'O', 'N', 'N', 4]
['ARG', 'CB', 'C', 'N', 'N', 5]
['ARG', 'CG', 'C', 'N', 'N', 6]
['ARG', 'CD', 'C', 'N', 'N', 7]
['ARG', 'NE', 'N', 'N', 'N', 8]
['ARG', 'CZ', 'C', 'N', 'N', 9]
['ARG', 'NH1', 'N', 'N', 'N', 10]
['ARG', 'NH2', 'N', 'N', 'N', 11]
['ARG', 'OXT', 'O', 'N', 'N', 12]
['ARG', 'H', 'H', 'N', 'N', 13]
['ARG', 'H2', 'H', 'N', 'N', 14]
['ARG', 'HA', 'H', 'N', 'N', 15]
['ARG', 'HB2', 'H', 'N', 'N', 16]
['ARG', 'HB3', 'H', 'N', 'N', 17]
['ARG', 'HG2', 'H', 'N', 'N', 18]
['ARG', 'HG3', 'H', 'N', 'N', 19]
['ARG', 'HD2', 'H', 'N', 'N', 20]
['ARG', 'HD3', 'H', 'N', 'N', 21]
['ARG', 'HE', 'H', 'N', 'N', 22]
['ARG', 'HH11', 'H', 'N', 'N', 23]
['ARG', 'HH12', 'H', 'N', 'N', 24]
['ARG', 'HH21', 'H', 'N', 'N', 25]
['ARG', 'HH22', 'H', 'N', 'N', 26]
['ARG', 'HXT', 'H', 'N', 'N', 27]
['ASN', 'N', 'N', 'N', 'N', 28]
['ASN', 'CA', 'C', 'N', 'S', 29]
['ASN', 'C', 'C', 'N', 'N', 30]
['ASN',

In [120]:
for ii in item.getObj('chem_comp_bond'):
    print(ii)

['ARG', 'N', 'CA', 'sing', 'N', 'N', 1]
['ARG', 'N', 'H', 'sing', 'N', 'N', 2]
['ARG', 'N', 'H2', 'sing', 'N', 'N', 3]
['ARG', 'CA', 'C', 'sing', 'N', 'N', 4]
['ARG', 'CA', 'CB', 'sing', 'N', 'N', 5]
['ARG', 'CA', 'HA', 'sing', 'N', 'N', 6]
['ARG', 'C', 'O', 'doub', 'N', 'N', 7]
['ARG', 'C', 'OXT', 'sing', 'N', 'N', 8]
['ARG', 'CB', 'CG', 'sing', 'N', 'N', 9]
['ARG', 'CB', 'HB2', 'sing', 'N', 'N', 10]
['ARG', 'CB', 'HB3', 'sing', 'N', 'N', 11]
['ARG', 'CG', 'CD', 'sing', 'N', 'N', 12]
['ARG', 'CG', 'HG2', 'sing', 'N', 'N', 13]
['ARG', 'CG', 'HG3', 'sing', 'N', 'N', 14]
['ARG', 'CD', 'NE', 'sing', 'N', 'N', 15]
['ARG', 'CD', 'HD2', 'sing', 'N', 'N', 16]
['ARG', 'CD', 'HD3', 'sing', 'N', 'N', 17]
['ARG', 'NE', 'CZ', 'sing', 'N', 'N', 18]
['ARG', 'NE', 'HE', 'sing', 'N', 'N', 19]
['ARG', 'CZ', 'NH1', 'sing', 'N', 'N', 20]
['ARG', 'CZ', 'NH2', 'doub', 'N', 'N', 21]
['ARG', 'NH1', 'HH11', 'sing', 'N', 'N', 22]
['ARG', 'NH1', 'HH12', 'sing', 'N', 'N', 23]
['ARG', 'NH2', 'HH21', 'sing', 'N', 